In [54]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

In [55]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import pickle

### Function Definitions

Function to fetch pages from wind power portal and close. This is a first essential step to gather all pages before progressing to actual scraping the links.

In [56]:
def fetch_num_pages(country):
    driver_path=r"C:\Program Files (x86)\chromedriver.exe"
    email_id="nipun.bhushan@gmail.com"
    password="FieldCedar08"
    login_url="https://www.thewindpower.net/login_en.php"
    base_url="https://www.thewindpower.net/windfarms_list_en.php"
    options = Options()
    options.add_argument('user-agent=fake-useragent')
    driver = webdriver.Chrome(options=options)
    actions = ActionChains(driver)
    driver.get(login_url)
    driver.maximize_window()
    driver.implicitly_wait(5)
    driver.find_element(By.XPATH,'//*[@id="bloc_texte"]/form/table/tbody/tr[2]/td/span/input').send_keys(email_id)
    driver.find_element(By.XPATH,'//*[@id="bloc_texte"]/form/table/tbody/tr[4]/td/input').send_keys(password)
    driver.find_element(By.XPATH,'//*[@id="bloc_texte"]/form/table/tbody/tr[6]/td/input').click()
    driver.implicitly_wait(5)
    driver.get(base_url)
    driver.find_element(By.LINK_TEXT,country).click()
    driver.implicitly_wait(5)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    page_change_elements = driver.find_elements(By.CLASS_NAME,"bx-pager-link")
    return [element.get_attribute("text") for element in page_change_elements]

Function to fetch to target windpower page

In [57]:
def fetch_to_windfarm_page(page_num,country):
    driver_path=r"C:\Program Files (x86)\chromedriver.exe"
    email_id="nipun.bhushan@gmail.com"
    password="FieldCedar08"
    login_url="https://www.thewindpower.net/login_en.php"
    base_url="https://www.thewindpower.net/windfarms_list_en.php"
    options = Options()
    options.add_argument('user-agent=fake-useragent')
    driver = webdriver.Chrome(options=options)
    actions = ActionChains(driver)
    driver.get(login_url)
    driver.maximize_window()
    driver.implicitly_wait(5)
    driver.find_element(By.XPATH,'//*[@id="bloc_texte"]/form/table/tbody/tr[2]/td/span/input').send_keys(email_id)
    driver.find_element(By.XPATH,'//*[@id="bloc_texte"]/form/table/tbody/tr[4]/td/input').send_keys(password)
    driver.find_element(By.XPATH,'//*[@id="bloc_texte"]/form/table/tbody/tr[6]/td/input').click()
    driver.implicitly_wait(5)
    driver.get(base_url)
    driver.find_element(By.LINK_TEXT,country).click()
    driver.implicitly_wait(5)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    page_change_elements = driver.find_elements(By.CLASS_NAME,"bx-pager-link")
    page_numbers = [element.get_attribute("text") for element in page_change_elements]

    if page_num=='1':
        page_change_elements[0].click()
        driver.implicitly_wait(5)
    else:
        for element in page_change_elements[1:]:
            if element.get_attribute("text")==page_num:
                element.click()
                driver.implicitly_wait(5)

    return driver

Function to fetch windfarm links from each page

In [58]:
def fetch_windfarms_from_page(driver,page_num,country):
    windfarm_links=[]
    rows=driver.find_elements(By.XPATH,'//*[@id="bloc_texte"]/div[1]/div/div[1]/ul/li['+page_num+']/table/tbody/tr')
    for row in rows[2:(len(rows)-1)]:
        if row.find_elements(By.TAG_NAME, "td")[0].text=='':
            continue
        windfarm_links.append((row.find_elements(By.TAG_NAME, "td")[0].text, row.find_elements(By.TAG_NAME, "td")[0].find_element(By.TAG_NAME, "a").get_attribute('href')))
    return windfarm_links

Extract number of turbines value

In [59]:
def extract_num_turbines(text):
    match = re.search(r'\b(\d+)\b.*(turbines|turbine)', text)
    if match:
        number = int(match.group(1))
        return number
    else:
        return None

Extract load rate

In [60]:
def extract_load_rate(text):
    match = re.search(r'Expected load rate: \b([+-]?\d+\.\d+)\b.* %', text)
    if match:
        number = float(match.group(1))
        return number
    else:
        return None

Extract expected production value

In [61]:
def extract_expected_prod(text):
    match = re.search(r'Expected production: \b(\d{1,3}(,\d{3})*)\b.* kWh', text)
    if match:
        number = int(match.group(1).replace(',', ''))
        return number
    else:
        return None

This calls the above functions to extract meaningful data and calculate normalized wind power value per turbine

In [62]:
def extract_power_per_turbine(url):
    turbine_list=[]
    load_list=[]
    power_prod_list=[]
    lat_list=[]
    lon_list=[]
    response = requests.get(url)
    if response.status_code==200:
        soup = BeautifulSoup(response.content, "html.parser")
        for element in soup.find_all('li'):
            if ("turbines" in element.text.lower()) or ("turbine" in element.text.lower()):
                turbine_list.append(extract_num_turbines(element.text))
            if "Expected load rate" in element.text:
                load_list.append(extract_load_rate(element.text))
            if "Expected production" in element.text:
                power_prod_list.append(extract_expected_prod(element.text))
            if "Latitude" in element.text:
                lat_list.append(extract_lat(element.text))
            if "Longitude" in element.text:
                lon_list.append(extract_lon(element.text))
    turbine_list=[element for element in turbine_list if element is not None ]
    if len(load_list) == len(power_prod_list):
        turbine_list=turbine_list[:len(load_list)]
    if sum(turbine_list)==0 or len(load_list)==0 or len(lat_list)==0 or len(lon_list)==0:
        return None
    else:
        return ((lat_list[0],lon_list[0]),round(sum(power_prod_list)/sum(turbine_list),2))

Deduplicate Tuple list based on th e first element of every tuple

In [63]:
def deduplicate_tuple_list(tuple_list):
    tuple_list_a, tuple_list_b = zip(*tuple_list)
    df = pd.DataFrame({'tuple_list_a':list(tuple_list_a),'tuple_list_b':list(tuple_list_b)})
    df = df.drop_duplicates(subset=['tuple_list_a'], keep='first')
    return list(zip(list(df['tuple_list_a']), list(df['tuple_list_b'])))

Extract Latitude

In [64]:
def extract_lat(text):
    try:
        latitude = re.search("Latitude: (.*)", text).group(1)
    except:
        return None
    lat_parts = latitude.split(" ")
    degrees = float(lat_parts[0].replace("Â°", ""))
    minutes = float(lat_parts[1].replace("'", ""))
    seconds = float(lat_parts[2].replace("\"", ""))
    
    latitude = degrees + minutes/60 + seconds/3600
    
    return round(latitude,5)

Extract Longitude

In [65]:
def extract_lon(text):
    try:
        longitude = re.search("Longitude: (.*)", text).group(1)
    except:
        return None
    lng_parts = longitude.split(" ")
    degrees = float(lng_parts[0].replace("Â°", ""))
    minutes = float(lng_parts[1].replace("'", ""))
    seconds = float(lng_parts[2].replace("\"", ""))

    longitude = degrees + minutes/60 + seconds/3600
    
    return round(longitude,5)

### Execution Block to extract all links to individual wind farm power output stats

In [66]:
all_winfarm_links=[]
countries=['Dominica' , 'Dominican Republic' , 'El Salvador' , 'Grenada' , 'Guatemala' , 'Honduras' , 'Jamaica' , 'Mexico' , 'Nicaragua' , 'Panama' , 'Puerto Rico' , 'Saint Kitts and Nevis' , 'The Bahamas' , 'USA' , 'Argentina' , 'Bolivia' , 'Brazil' , 'Chile' , 'Colombia' , 'Curaçao' , 'Ecuador' , 'Guyana' , 'Peru' , 'Uruguay' , 'Venezuela' , 'Albania' , 'Austria' , 'Belarus' , 'Belgium' , 'Bosnia and Herzegovina' , 'Bulgaria' , 'Croatia' , 'Cyprus' , 'Czech Republic' , 'Denmark' , 'Estonia' , 'Faroe Islands' , 'Finland' , 'France' , 'Germany' , 'Greece' , 'Hungary' , 'Iceland' , 'Italy' , 'Kosovo' , 'Latvia' , 'Lithuania' , 'Luxembourg' , 'Montenegro' , 'Netherlands' , 'North Macedonia' , 'Norway' , 'Poland' , 'Portugal' , 'Romania' , 'Serbia' , 'Slovakia' , 'Slovenia' , 'Spain' , 'Sweden' , 'Switzerland' , 'Ukraine' , 'United-Kingdom' , 'Algeria' , 'Cape Verde' , 'Chad' , 'Djibouti' , 'Egypt' , 'Eritrea' , 'Ethiopia' , 'Gambia' , 'Ghana' , 'Kenya' , 'Libya' , 'Mauritania' , 'Mauritius' , 'Morocco' , 'Mozambique' , 'Namibia' , 'Nigeria' , 'Senegal' , 'Seychelles' , 'South Africa' , 'Sudan' , 'Tanzania' , 'Tunisia' , 'Uganda' , 'Armenia' , 'Azerbaijan' , 'Bahrain' , 'Bangladesh' , 'Bhutan' , 'Cambodia' , 'China' , 'Georgia' , 'India' , 'Iran' , 'Israel' , 'Japan' , 'Jordan' , 'Kazakhstan' , 'Kuwait' , 'Mongolia' , 'Oman' , 'Pakistan' , 'Russia' , 'Saudi Arabia' , 'Singapore' , 'South Korea' , 'Sri Lanka' , 'Syria' , 'Taiwan' , 'Thailand' , 'Turkey' , 'United Arab Emirates' , 'Uzbekistan' , 'Vietnam' , 'Australia' , 'Fiji' , 'Guam' , 'Indonesia' , 'Micronesia' , 'New-Zealand' , 'Philippines' , 'Samoa' , 'Tonga' , 'Vanuatu']
for country in countries:
    print("Processing for country: ", country, "..")
    page_numbers = fetch_num_pages(country)
    for page_num in page_numbers:
        print("Fetching Page: ",page_num,"..")
        driver=fetch_to_windfarm_page(page_num,country)
        all_winfarm_links.append(fetch_windfarms_from_page(driver,page_num,country))
        print("Completed fetching Page: ", page_num)
        driver.close()
all_winfarm_links = [winfarm_link for winfarm_links in all_winfarm_links for winfarm_link in winfarm_links]
all_winfarm_links = deduplicate_tuple_list(all_winfarm_links)
pickle.dump(all_winfarm_links, open('all_winfarm_links.p', 'wb'))

Processing for country:  Dominica ..
Fetching Page:  1 ..
Completed fetching Page:  1
Processing for country:  Dominican Republic ..
Fetching Page:  1 ..
Completed fetching Page:  1
Processing for country:  El Salvador ..
Fetching Page:  1 ..
Completed fetching Page:  1
Processing for country:  Grenada ..
Fetching Page:  1 ..
Completed fetching Page:  1
Processing for country:  Guatemala ..
Fetching Page:  1 ..
Completed fetching Page:  1
Processing for country:  Honduras ..
Fetching Page:  1 ..
Completed fetching Page:  1
Processing for country:  Jamaica ..
Fetching Page:  1 ..
Completed fetching Page:  1
Processing for country:  Mexico ..
Fetching Page:  1 ..
Completed fetching Page:  1
Fetching Page:  2 ..
Completed fetching Page:  2
Processing for country:  Nicaragua ..
Fetching Page:  1 ..
Completed fetching Page:  1
Processing for country:  Panama ..
Fetching Page:  1 ..
Completed fetching Page:  1
Processing for country:  Puerto Rico ..
Fetching Page:  1 ..
Completed fetching Pa

Completed fetching Page:  28
Fetching Page:  29 ..
Completed fetching Page:  29
Fetching Page:  30 ..
Completed fetching Page:  30
Fetching Page:  31 ..
Completed fetching Page:  31
Fetching Page:  32 ..
Completed fetching Page:  32
Fetching Page:  33 ..
Completed fetching Page:  33
Fetching Page:  34 ..
Completed fetching Page:  34
Fetching Page:  35 ..
Completed fetching Page:  35
Fetching Page:  36 ..
Completed fetching Page:  36
Fetching Page:  37 ..
Completed fetching Page:  37
Fetching Page:  38 ..
Completed fetching Page:  38
Fetching Page:  39 ..
Completed fetching Page:  39
Fetching Page:  40 ..
Completed fetching Page:  40
Processing for country:  Estonia ..
Fetching Page:  1 ..
Completed fetching Page:  1
Fetching Page:  2 ..
Completed fetching Page:  2
Processing for country:  Faroe Islands ..
Fetching Page:  1 ..
Completed fetching Page:  1
Processing for country:  Finland ..
Fetching Page:  1 ..
Completed fetching Page:  1
Fetching Page:  2 ..
Completed fetching Page:  2


Fetching Page:  1 ..
Completed fetching Page:  1
Fetching Page:  2 ..
Completed fetching Page:  2
Fetching Page:  3 ..
Completed fetching Page:  3
Fetching Page:  4 ..
Completed fetching Page:  4
Fetching Page:  5 ..
Completed fetching Page:  5
Fetching Page:  6 ..
Completed fetching Page:  6
Fetching Page:  7 ..
Completed fetching Page:  7
Fetching Page:  8 ..
Completed fetching Page:  8
Fetching Page:  9 ..
Completed fetching Page:  9
Processing for country:  Portugal ..
Fetching Page:  1 ..
Completed fetching Page:  1
Fetching Page:  2 ..
Completed fetching Page:  2
Fetching Page:  3 ..
Completed fetching Page:  3
Fetching Page:  4 ..
Completed fetching Page:  4
Fetching Page:  5 ..
Completed fetching Page:  5
Fetching Page:  6 ..
Completed fetching Page:  6
Fetching Page:  7 ..
Completed fetching Page:  7
Fetching Page:  8 ..
Completed fetching Page:  8
Fetching Page:  9 ..
Completed fetching Page:  9
Processing for country:  Romania ..
Fetching Page:  1 ..
Completed fetching Page:

WebDriverException: Message: chrome not reachable
Stacktrace:
Backtrace:
	(No symbol) [0x004A6643]
	(No symbol) [0x0043BE21]
	(No symbol) [0x0033D960]
	(No symbol) [0x0033295A]
	(No symbol) [0x0036231E]
	(No symbol) [0x0035D630]
	(No symbol) [0x0035A899]
	(No symbol) [0x00396917]
	(No symbol) [0x0039655C]
	(No symbol) [0x0038FB76]
	(No symbol) [0x003649C1]
	(No symbol) [0x00365E5D]
	GetHandleVerifier [0x0071A142+2497106]
	GetHandleVerifier [0x007485D3+2686691]
	GetHandleVerifier [0x0074BB9C+2700460]
	GetHandleVerifier [0x00553B10+635936]
	(No symbol) [0x00444A1F]
	(No symbol) [0x0044A418]
	(No symbol) [0x0044A505]
	(No symbol) [0x0045508B]
	BaseThreadInitThunk [0x76067D69+25]
	RtlInitializeExceptionChain [0x773ABB9B+107]
	RtlClearBits [0x773ABB1F+191]


In [71]:
all_winfarm_links = [winfarm_link for winfarm_links in all_winfarm_links for winfarm_link in winfarm_links]
all_winfarm_links = deduplicate_tuple_list(all_winfarm_links)

### Execution Block to extract power stats for individual wind farm

In [72]:
windfarm_name_list=[]
lat_list=[]
lon_list=[]
windfarms_power_stats=[]
for windfarm_link in all_winfarm_links:
    print("Processing: ",windfarm_link[0],"..")
    if extract_power_per_turbine(windfarm_link[1])!=None:
        output = extract_power_per_turbine(windfarm_link[1])
        lat_lon = output[0]
        power = output[1]
        windfarm_name_list.append(windfarm_link[0])
        lat_list.append(lat_lon[0])
        lon_list.append(lat_lon[1])
        windfarms_power_stats.append(power)
    
df_windfarms_power_stats = pd.DataFrame({'windfarm_name':windfarm_name_list,'latitude':lat_list,'longitude':lon_list,'windfarm_power':windfarms_power_stats})
df_windfarms_power_stats=df_windfarms_power_stats[~df_windfarms_power_stats['windfarm_power'].isna()]
df_windfarms_power_stats.to_csv("output_files/windfarms_power_stats.csv",index=False)

Processing:   Rosalie Bay ..
Processing:   Agua Clara ..
Processing:   El Guanillo ..
Processing:   Granadillos ..
Processing:   Larimar ..
Processing:   Larimar II ..
Processing:   Los Cocos-Quilvio Cabrera ..
Processing:   Los Guzmancitos ..
Processing:   Matafongo ..
Processing:   Puerto Plata-Imbert ..
Processing:   Ventus (SV) ..
Processing:   Grenade ..
Processing:   Las Cumbres ..
Processing:   San Antonio ..
Processing:   Viento Blanco ..
Processing:   Cerro de Hula ..
Processing:   Chinchayote ..
Processing:   Ocean View ..
Processing:   San Marcos de Colón ..
Processing:   Malvern ..
Processing:   Munro College ..
Processing:   Munro Wind Farm ..
Processing:   Petroleum Corporation of Jamaica offshore wind farm ..
Processing:   Wigton ..
Processing:   Wigton II ..
Processing:   Wigton III ..
Processing:   Amistad ..
Processing:   Arriaga ..
Processing:   Bii Hioxio ..
Processing:   Bii Nee Stipa I ..
Processing:   Bii Nee Stipa II ..
Processing:   Bii Nee Stipa III ..
Process

Processing:   Blackwell Wind ..
Processing:   Blazing Star Wind Farm ..
Processing:   Block Island Wind Farm ..
Processing:   Bloom ..
Processing:   Blooming Grove ..
Processing:   Blue Breezes I & II ..
Processing:   Blue Canyon ..
Processing:   Blue Canyon II ..
Processing:   Blue Canyon V ..
Processing:   Blue Canyon VI ..
Processing:   Blue Cloud ..
Processing:   Blue Creek ..
Processing:   Blue Sky Green Field ..
Processing:   Blue Summit ..
Processing:   Blue Summit II ..
Processing:   Blue Summit III ..
Processing:   Bluegrass Ridge wind energy project ..
Processing:   Bluestem ..
Processing:   Bluestone wind farm ..
Processing:   Bluff Point ..
Processing:   BMP wind farm ..
Processing:   Bobcat Bluff ..
Processing:   Boiling Springs ..
Processing:   Bolton Valley Resort ..
Processing:   Border Winds ..
Processing:   Borderlands ..
Processing:   Boulder NREL Wind (dismantled) ..
Processing:   Boulder NREL Wind ..
Processing:   Bowling Green Municipal ..
Processing:   Boyce Dair

Processing:   Dakota Range Wind I & II ..
Processing:   Danielson Wind ..
Processing:   Day County ..
Processing:   Deerfield ..
Processing:   Deerfield II ..
Processing:   Deerfield Wind (dismantled) ..
Processing:   Deerfield Wind ..
Processing:   Delaware Mountain (dismantled) ..
Processing:   Delta Junction ..
Processing:   Department of Correction - NCCI Gardner ..
Processing:   Desert Sky Wind Farm (dismantled) ..
Processing:   Desert Sky Wind Farm Repowering ..
Processing:   Deuel ..
Processing:   DeWind Demonstration Project ..
Processing:   DH Blattner Headquarters ..
Processing:   Diablo Winds ..
Processing:   Diamond Spring Wind ..
Processing:   Diamond Trail ..
Processing:   Diamond Trail Expansion ..
Processing:   Diamond Vista ..
Processing:   Diamond Willow extension ..
Processing:   Diamond Willow Wind (07) ..
Processing:   Diamond Willow Wind (08) ..
Processing:   DifWind I ..
Processing:   DifWind II ..
Processing:   DifWind IV ..
Processing:   Difwind IX ..
Processin

Processing:   Harvest Wind 2 ..
Processing:   Harvest Wind Farm ..
Processing:   Hasslebach Meats ..
Processing:   Hatchett Ridge ..
Processing:   Haviland ..
Processing:   Hawai Renewable ..
Processing:   Hawkeye ..
Processing:   Hay Canyon ..
Processing:   Haystack ..
Processing:   Headwaters ..
Processing:   Headwaters II ..
Processing:   Heart of Texas ..
Processing:   Heartland College ..
Processing:   Heartland Divide ..
Processing:   Helena wind farm ..
Processing:   Helzel & Schwarzhoff 88 ..
Processing:   Hereford I ..
Processing:   Heritage Flight ..
Processing:   Heritage Garden ..
Processing:   Hidalgo II ..
Processing:   High Lonesome (NM) ..
Processing:   High Lonesome (TX) ..
Processing:   High Mesa ..
Processing:   High Plains ..
Processing:   High Plains Tech Center ..
Processing:   High Prairie ..
Processing:   High Sheldon ..
Processing:   High Winds ..
Processing:   Highland ..
Processing:   Highland II ..
Processing:   Highland North ..
Processing:   Highland Wind 

Processing:   Loraine ..
Processing:   Loranger Power Generation ..
Processing:   Lorenzo Winds ..
Processing:   Los Mirasoles ..
Processing:   Los Vientos I ..
Processing:   Los Vientos II ..
Processing:   Los Vientos III ..
Processing:   Los Vientos IV ..
Processing:   Los Vientos V ..
Processing:   Lost Creek Ridge Wind Farm ..
Processing:   Lost Lakes Wind Farm ..
Processing:   Lower Snake River I ..
Processing:   Lubbock Wind Ranch I ..
Processing:   Lund Hill ..
Processing:   Lundgren ..
Processing:   Luther College Turbine ..
Processing:   Luverne ..
Processing:   LWSC Wind Turbine ..
Processing:   Macho Springs ..
Processing:   Mackinaw City ..
Processing:   Macksburg ..
Processing:   Madison Wind Power Project ..
Processing:   Magic Valley I ..
Processing:   Maiden Winds ..
Processing:   Maine Research Array ..
Processing:   Majestic ..
Processing:   Mammoth Plains ..
Processing:   Mann Packing ..
Processing:   Manzana ..
Processing:   Maple Ridge ..
Processing:   Maple Ridge 

...

Processing:   São Macario II ..
Processing:   Sao Mamede ..
Processing:   Sao Paio ..
Processing:   Sao Pedro ..
Processing:   Sardinha ..
Processing:   Sebolido ..
Processing:   Seixinhos ..
Processing:   Senhora da Vitoria ..
Processing:   Senhora do Castelo ..
Processing:   Senhora do Socorro ..
Processing:   Seramena ..
Processing:   Sernancelhe ..
Processing:   Serra Alta ..
Processing:   Serra Branca (dismantled) ..
Processing:   Serra Branca II (dismantled) ..
Processing:   Serra da Amêndoa ..
Processing:   Serra da Boa Viagem ..
Processing:   Serra da Boneca ..
Processing:   Serra da Boneca - Torrão ..
Processing:   Serra da Cabreira ..
Processing:   Serra da Capucha ..
Processing:   Serra da Escusa ..
Processing:   Serra da Lage ..
Processing:   Serra de Alvoaça ..
Processing:   Serra de Todo-o-Mundo ..
Processing:   Serra do Alvao ..
Processing:   Serra do Barroso ..
Processing:   Serra do Barroso II ..
Processing:   Serra do Barroso III ..
Processing:   Serra do Cume ..
Proc

Processing:   Barrax Norte Sur ..
Processing:   Becerril ..
Processing:   Becerril II ..
Processing:   Belchite ..
Processing:   Belmonte ..
Processing:   Benalaz I ..
Processing:   Benalaz II ..
Processing:   Bermejo ..
Processing:   Bidueiros Fase I ..
Processing:   Biscay Marine Energy Park ..
Processing:   Boira ..
Processing:   Bolaños ..
Processing:   Boquerón ..
Processing:   Boquerón I ..
Processing:   Boquerón II ..
Processing:   Boquerón III ..
Processing:   Bordecorex Norte ..
Processing:   Borja I ..
Processing:   Borja II (Arbolitas) ..
Processing:   Bosque Alto ..
Processing:   Boyal I ..
Processing:   Boyal II ..
Processing:   Boyal III ..
Processing:   Breña ..
Processing:   Brulles ..
Processing:   Buenavista ..
Processing:   Buio ..
Processing:   Bullana ..
Processing:   Buniel ..
Processing:   Bureba ..
Processing:   Buseco ..
Processing:   Bustelo ..
Processing:   Cabanillas II ..
Processing:   Cabeza del Conde ..
Processing:   Cabeza Mesa ..
Processing:   Cabeza Mo

Processing:   El Pilar ..
Processing:   El Pino ..
Processing:   El Poleo ..
Processing:   El Pradillo ..
Processing:   El Puerto (Unificado) ..
Processing:   El Pulpal ..
Processing:   El Puntal ..
Processing:   El Raso ..
Processing:   El Rebollar ..
Processing:   El Redondal Fase 1 ..
Processing:   El Redondal Fase 2 ..
Processing:   El Relumbrar ..
Processing:   El Rincón ..
Processing:   El Rodeo ..
Processing:   El Romeral ..
Processing:   El Ruedo ..
Processing:   El Sardón ..
Processing:   El Saso ..
Processing:   El Saucito ..
Processing:   El Segredal ..
Processing:   El Sotillo-Tablares ..
Processing:   El Tablado ..
Processing:   El Teruelo ..
Processing:   El Tesorillo ..
Processing:   El Toranzo ..
Processing:   El Toranzo Ampliación ..
Processing:   El Valle Valdenavarro ..
Processing:   El Venzo ..
Processing:   El Viudo I ..
Processing:   El Viudo II ..
Processing:   Elgea ..
Processing:   Elgea Ampliación ..
Processing:   Elgea-Urkilla ..
Processing:   Elican ..
Proce

Processing:   Las Viñas (Fase B) ..
Processing:   Layna ..
Processing:   Leboreiro ..
Processing:   Lecrin ..
Processing:   Leitza-Beruete ..
Processing:   Leoz ..
Processing:   Lerga ..
Processing:   Les Calobres ..
Processing:   Les Colladetes ..
Processing:   Les Comes ..
Processing:   Les Forques ..
Processing:   Les Forques II ..
Processing:   Les Rotes ..
Processing:   Leste ..
Processing:   Levantera ..
Processing:   Lezuza ..
Processing:   Llanos de Juan Grande ..
Processing:   Llanos de la Aldea ..
Processing:   Llanos de la Esquina ..
Processing:   Llanos de San Martin ..
Processing:   Llanos del Espino ..
Processing:   Lodoso ..
Processing:   Loma de Almendarache ..
Processing:   Loma de Ayala ..
Processing:   Loma de LAzaro ..
Processing:   Loma De Los Pinos ..
Processing:   Loma del Capón ..
Processing:   Loma del Suyal (Fase A) ..
Processing:   Loma del Suyal (Fase B) ..
Processing:   Loma Gorda (Guadalajara) ..
Processing:   Loma Gorda (Zaragoza) ..
Processing:   Loma Vi

Processing:   Pena Forcada ..
Processing:   Pena Forcada-Catasol II ..
Processing:   Peña Galluda ..
Processing:   Pena Grande ..
Processing:   Peña I ..
Processing:   Peña II ..
Processing:   Pena Luisa ..
Processing:   Peña Mijan ..
Processing:   Peña Nebina ..
Processing:   Peña Roldana ..
Processing:   Pena Ventosa ..
Processing:   Pena Ventosa Reformado ..
Processing:   Peñaflor III ..
Processing:   Peñaflor IV ..
Processing:   Peñas de Dios I ..
Processing:   Peñas de Dios II ..
Processing:   Penas Grandes ..
Processing:   Penouta ..
Processing:   Perdiguera ..
Processing:   PES Cerceda ..
Processing:   PESUR (dismantled) ..
Processing:   PESUR (Repotenciado) ..
Processing:   Picador ..
Processing:   Picazo ..
Processing:   Pico Collalbas ..
Processing:   Pico Gallo ..
Processing:   Piedras del Alto ..
Processing:   Piletas 1 ..
Processing:   Pivot Buoy - PLOCAN ..
Processing:   Pla d´Embalagué ..
Processing:   Plana de Jarreta ..
Processing:   Plana de la Balsa ..
Processing:   